# Imports

In [9]:
import tensorflow as tf

In [10]:
import pandas as pd

In [11]:
import numpy as np

In [8]:
# import pertinent libraries
import os
import sys
import datetime
import glob as glob
import numpy as np
import tensorflow as tf
import keras

# [Keras Models]
# import the Keras implementations of VGG16, VGG19, InceptionV3 and Xception models
# the model used here is VGG16
from keras.utils import image_dataset_from_directory
from keras.applications import vgg16
from keras import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Function to create dataset (you can place this here or in a separate cell)
def create_dataset(directory, batch_size, image_size=(224, 224)):
    return image_dataset_from_directory(
        directory,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True
    )

In [9]:
import keras
from keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Dense
from keras.applications.resnet50 import ResNet50, preprocess_input

In [10]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric

class F1Score(Metric):
    def __init__(self, num_classes, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', initializer='zeros', shape=(num_classes,))
        self.false_positives = self.add_weight(name='fp', initializer='zeros', shape=(num_classes,))
        self.false_negatives = self.add_weight(name='fn', initializer='zeros', shape=(num_classes,))

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(tf.argmax(y_true, axis=1), self.num_classes)
        y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), self.num_classes)
        
        self.true_positives.assign_add(tf.reduce_sum(y_true * y_pred, axis=0))
        self.false_positives.assign_add(tf.reduce_sum((1 - y_true) * y_pred, axis=0))
        self.false_negatives.assign_add(tf.reduce_sum(y_true * (1 - y_pred), axis=0))

    def result(self):
        precision = tf.reduce_sum(self.true_positives) / (tf.reduce_sum(self.true_positives) + tf.reduce_sum(self.false_positives) + tf.keras.backend.epsilon())
        recall = tf.reduce_sum(self.true_positives) / (tf.reduce_sum(self.true_positives) + tf.reduce_sum(self.false_negatives) + tf.keras.backend.epsilon())
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.true_positives.assign(tf.zeros_like(self.true_positives))
        self.false_positives.assign(tf.zeros_like(self.false_positives))
        self.false_negatives.assign(tf.zeros_like(self.false_negatives))

In [11]:
import cv2

# Data

## Data Augmentation

In [16]:
import Augmentor
import os

# Get the current working directory
current_dir = os.getcwd()

# Create the relative path
relative_path = os.path.join(current_dir, "archive/train")

# Create an Augmentor pipeline
p = Augmentor.Pipeline(relative_path)

# Add operations to the pipeline
p.rotate90(probability=0.7)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)
p.random_brightness(probability=0.3, min_factor=0.8, max_factor=1.2)
p.random_contrast(probability=0.3, min_factor=0.8, max_factor=1.2)
p.random_distortion(probability=0.3, grid_width=4, grid_height=4, magnitude=8)

# Calculate the number of original images
original_image_count = len(p.augmentor_images)

# Calculate how many times we need to multiply the original dataset
multiplication_factor = 10000 // original_image_count

# Generate augmented images
p.sample(10000)

print(f"Generated {10000} augmented images.")

Initialised with 600 image(s) found.
Output directory set to c:\Users\BPIT\Desktop\minor-project-main\MinorProject\archive/train\output.

Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x204B4410D90>: 100%|██████████| 10000/10000 [00:16<00:00, 591.73 Samples/s]                  


Generated 10000 augmented images.


## Bilateral Filtering

In [17]:
def Bilateral_filtering(directory):
    for filename in os.listdir(directory):       
        # load the image
        img = cv2.imread(os.path.join(directory, filename))

        # apply bilateral filtering
        filtered_img = cv2.bilateralFilter(img, 6, 75,1475)

        # save the filtered image
        cv2.imwrite(os.path.join(directory,filename), filtered_img)
    

### Train

In [18]:
directory = r"archive\valid\Positive"
Bilateral_filtering(directory)
directory = r"archive\valid\Positive"
Bilateral_filtering(directory)

In [19]:
directory = r"archive\valid\Negative"
Bilateral_filtering(directory)

### Val and Test

In [20]:
directory = r"archive\valid\Positive"
Bilateral_filtering(directory)

directory = r"archive\valid\Negative"
Bilateral_filtering(directory)

directory = r"archive\valid\Positive"
Bilateral_filtering(directory)

directory = r"archive\valid\Negative"
Bilateral_filtering(directory)


## Grayscaling

In [21]:
def Grayscaling(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(filepath, gray_image)

### Train

In [22]:
directory = r"archive\valid\Positive"
Grayscaling(directory)
directory = r"archive\valid\Negative"
Grayscaling(directory)

### Val and Test

In [23]:
directory = r"archive\valid\Positive"
Grayscaling(directory)
directory = r"archive\valid\Negative"
Grayscaling(directory)

directory = r"archive\valid\Positive"
Grayscaling(directory)
directory = r"archive\valid\Negative"
Grayscaling(directory)

# Data Generators

In [24]:
negative_images = os.listdir(r"archive\valid\Negative")
positive_images = os.listdir(r"archive\valid\Positive")

In [25]:
num_classes = 2
image_resize = 224
batch_size_training = 50
batch_size_validation = 50

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [27]:
train_generator = data_generator.flow_from_directory(
    r"archive\train\output",  # Use raw string to avoid escape sequences
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


Found 26639 images belonging to 2 classes.


In [28]:
validation_generator = data_generator.flow_from_directory(
    r"archive\valid",
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 200 images belonging to 2 classes.


# Model

In [29]:
model = Sequential()
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))


NameError: name 'num_classes' is not defined

In [30]:
model.add(Dense(2, activation = "sigmoid"))

In [31]:
model.layers[0].trainable = False

In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,816 (90.00 MB)

 Trainable params: 4,104 (16.03 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', F1Score(num_classes=2)])

In [34]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 20

In [36]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/3
533/533 ━━━━━━━━━━━━━━━━━━━━ 791s 1s/step - accuracy: 0.9983 - f1_score: 0.9983 - loss: 0.1483 - val_accuracy: 0.9900 - val_f1_score: 0.9900 - val_loss: 0.1175
Epoch 2/3
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step - accuracy: 0.0000e+00 - f1_score: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/3
533/533 ━━━━━━━━━━━━━━━━━━━━ 812s 2s/step - accuracy: 0.9999 - f1_score: 0.9999 - loss: 0.0832 - val_accuracy: 0.9900 - val_f1_score: 0.9900 - val_loss: 0.0809


In [45]:
data_generator_test = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
test_generator = data_generator_test.flow_from_directory(
    r'archive/test',
    target_size=(image_resize, image_resize),
    batch_size = batch_size_validation,
    class_mode='categorical',
    shuffle=False)


Found 200 images belonging to 2 classes.


In [48]:
model.evaluate(test_generator)

c:\Users\BPIT\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0574


[0.05762780085206032, 1.0, 1.0]

In [49]:
model.save('resnet_model1.h5')

# Model 2

In [6]:
from keras.applications.vgg16 import VGG16


In [7]:
model_vgg = Sequential()
model_vgg.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model_vgg.add(Dense(num_classes, activation='softmax'))
model_vgg.add(Dense(num_classes, activation ='sigmoid'))

NameError: name 'Sequential' is not defined

In [52]:
model_vgg.layers[0].trainable = False

In [53]:
model_vgg.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         1,026 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,720 (56.14 MB)

 Trainable params: 1,032 (4.03 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [54]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', F1Score(num_classes=2)])

In [ ]:
fit_history_vgg = model_vgg.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=4,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
    
)

Epoch 1/4
283/533 ━━━━━━━━━━━━━━━━━━━━ 9:35 2s/step - accuracy: 0.5067 - f1_score: 0.5067 - loss: 0.6390

In [181]:
model_vgg.save('vgg16_model1.h5')

In [ ]:
model_vgg.evaluate_generator(test_generator,steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

In [3]:
from keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    f1_val = 2*((precision_val*recall_val)/(precision_val+recall_val+K.epsilon()))
    return f1_val




In [5]:
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',recall,precision,f1_score])


NameError: name 'model' is not defined

### An array of [Loss, Accuracy, Recall, Precision, F1 Score]

In [ ]:
model.evaluate_generator(test_generator,steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

### Prediction for cases

In [205]:
directory = r"archive\valid\Positive"
Bilateral_filtering(directory)
Grayscaling(directory)


In [ ]:
from IPython.display import Image
Image(filename='E:\\Bali\\Kaggle Competition\\predict\\IMG_1129.JPG') 

In [210]:
img = image.load_img("E:\\Bali\\Kaggle Competition\\predict\\IMG_1129.JPG", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [ ]:
model = keras.models.load_model('resnet_model1.h5')

# Make a prediction
predictions = model.predict(x)

# Decode the predictions
# Load the class labels
class_labels = [ "Negative","Positive"]

# Get the index of the class with the highest probability
top_class_index = np.argmax(predictions)

# Get the class label
top_class_label = class_labels[top_class_index]
print(top_class_label)





